<a href="https://colab.research.google.com/github/nilesh0109/ML_SoSe19/blob/master/ML_ex07.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np
import matplotlib.pyplot as plt
import scipy

In [0]:
class K_Arm_Bandit:
  def __init__(self, num_arms):
    self.k = num_arms
    self.a = np.arange(num_arms, dtype=np.int16)
    self.N_a = np.zeros(num_arms, dtype=np.int16)
    self.Q = np.zeros(num_arms, dtype=np.float16)
    self.reset()
  
  def reset(self):
    '''This r_dist_means gives us the mean of the Normal reward distribution which is in itself comes from a Normal distrubition with zero mean'''
    self.r_dist_means = np.random.normal(size=10)
  
  def getReward(self, act):
    ''' Fetches the reward from a normal distribution given the action index'''
    return np.random.normal(self.r_dist_means[act])
  
  def get_rewards(act):
    rand_mean = np.random.normal()
    reward = np.random.normal(rand_mean)
    return reward
  
  def play_epsilon_greedy(self, epsilon):
    p = np.random.random()
    if p > epsilon:
      chosen_action = np.argmax(self.Q)
    else:
      chosen_action = np.random.choice(self.a)
    self.step(chosen_action)
  
  def step(self, act):
    self.N_a[act] += 1
    received_reward = self.getReward(act)
    self.Q[act] +=  (1 / self.N_a[act]) * (received_reward - self.Q[act])
    #print('received action= {:d} received_reward ={:f}'.format(act, received_reward))
    #print(self.Q)
      
    
    

In [74]:
testBed = K_Arm_Bandit(10)
print(testBed.N_a)
print(testBed.Q)

[0 0 0 0 0 0 0 0 0 0]
[0. 0. 0. 0. 0. 0. 0. 0. 0. 0.]


In [75]:
for i in range(10000):
  testBed.play_epsilon_greedy(1)
print(testBed.N_a)

[1026 1057  993 1010 1029  976 1023 1005  959  922]
